In [1]:
import torch
import numpy as np
from numpy import asarray
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch import permute
from PIL import Image
import matplotlib.pyplot as plt
import cv2
import os
from pathlib import Path
import pickle
import pandas as pd
from tqdm import tqdm

import sys
sys.path.insert(0,'../')
import label_tools as lt

#### Loading Files

load pre processed files

In [2]:
#imgs_path = Path('/home/richard/data/Schiefer/feature_extraction/clips/preprocessed_224_clipped_pred_polygon_224/')
#imgs_path = Path('/home/richard/data/Schiefer/feature_extraction/clips/preprocessed_224_clipped_pred_square_205/')
#imgs_path = Path('/home/richard/data/Schiefer/feature_extraction/clips/preprocessed_224_clipped_gt_polygon_296/')
#imgs_path = Path(''/home/richard/data/Schiefer/feature_extraction/clips/preprocessed_224_clipped_gt_square_296/'')

imgs_path = Path('/home/richard/data/Schiefer/combine/preprocessed_224_clipped_pred_polygon_1126/')
assert imgs_path.is_dir()
files = sorted(imgs_path.glob('*.png'))

randomizer = np.random.RandomState(seed=99833)
randomizer.shuffle(files)

assert len(files) == int(str(imgs_path).split('_')[5]) # all files are found
print("First 10 files are: {}".format([x.name for x in files[:10]]))

First 10 files are: ['CFB167_ortho_mask_0138_12_preprocessed.png', 'CFB151_ortho_mask_0105_12_preprocessed.png', 'CFB167_ortho_mask_0202_12_preprocessed.png', 'CFB151_ortho_mask_0254_12_preprocessed.png', 'CFB184_ortho_mask_0173_12_preprocessed.png', 'CFB114_ortho_mask_0041_10_preprocessed.png', 'CFB151_ortho_mask_0245_12_preprocessed.png', 'CFB151_ortho_mask_0184_12_preprocessed.png', 'CFB133_ortho_mask_0147_4_preprocessed.png', 'CFB151_ortho_mask_0109_12_preprocessed.png']


#### Loading Images

In [3]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()

def alter_image(img_name):
    image = Image.open(img_name)
    image_tensor = normalize(to_tensor(image)).unsqueeze(0)
    image_tensor = image_tensor.reshape(1,3,224,224)
    return image_tensor

def load_images_as_tensors(paths):
    images = [alter_image(image) for image in paths]
    return images

image_tensors = load_images_as_tensors(files)
assert len(image_tensors) == int(str(imgs_path).split('_')[5])

#### Extracting labels from filenames

In [4]:
def extract_labels(files): return [filename.stem.split('_')[4] for filename in files]
labels = extract_labels(files)
print('first 10 labels: {}'.format(labels[:10]))

first 10 labels: ['12', '12', '12', '12', '12', '10', '12', '12', '4', '12']


In [5]:
print(set(labels))

{'6', '12', '10', '8', '5', '4', '14', '11', '13'}


In [6]:
# change labels from number to species (of collab)
update = {
    '4':'Fagus_sylvatica',
    '5':'Fraxinus_excelsior',
    '6':'Quercus_spec',
    '8':'deadwood',
    '10':'Abies_alba',
    '11':'Larix_decidua',
    '12':'Picea_abies',
    '13':'Pinus_sylvestris',
    '14':'Pseudotsuga_menziesii'
}

updated_labels = (pd.Series(labels)).map(update)
species_labels = list(updated_labels)
labels = species_labels
print('first 10 labels: {}'.format(labels[:10]))

first 10 labels: ['Picea_abies', 'Picea_abies', 'Picea_abies', 'Picea_abies', 'Picea_abies', 'Abies_alba', 'Picea_abies', 'Picea_abies', 'Fagus_sylvatica', 'Picea_abies']


#### Label encoding

Standardize encodings of labels to make analysis easier afterwards.

In [7]:
le = lt.CustomLabelEncoder()
le.fit(labels, sorter=lambda x: x.upper())

labels_int = le.transform(labels[:10])
labels_str = le.inverse_transform(labels_int)

label_dir = Path('/home/richard/data/Schiefer/feature_extraction/labels')
filename = Path('ResNet152_polygon_pred_label_encodings_512_' + str(imgs_path).split('_')[5] + '.pickle')
with open(label_dir / filename, 'wb') as f:
    pickle.dump(le, f)

print('label encodings: {}'.format(le.mapper))
print('first 10 integer labels: {}'.format(labels_int))
print('first 10 string labels: {}'.format(labels_str))

label encodings: {'Abies_alba': 0, 'deadwood': 1, 'Fagus_sylvatica': 2, 'Fraxinus_excelsior': 3, 'Larix_decidua': 4, 'Picea_abies': 5, 'Pinus_sylvestris': 6, 'Pseudotsuga_menziesii': 7, 'Quercus_spec': 8}
first 10 integer labels: [5 5 5 5 5 0 5 5 2 5]
first 10 string labels: ['Picea_abies' 'Picea_abies' 'Picea_abies' 'Picea_abies' 'Picea_abies'
 'Abies_alba' 'Picea_abies' 'Picea_abies' 'Fagus_sylvatica' 'Picea_abies']


In [8]:
print(le.transform(labels))

[5 5 5 ... 7 7 0]


#### Feature Extraction

Load the EfficientNetV2 model

In [10]:
# Load the pretrained model
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = models.efficientnet_v2_l(weights=models.efficientnet_v2_l)# Use the model object to select the desired layer
layer = model._modules.get('avgpool')
# Set model to evaluation mode
model.eval()

/home/richard/mambaforge/envs/detectree2/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=EfficientNet_V2_L_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_V2_L_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_v2_l-59c71312.pth" to /home/richard/.cache/torch/hub/checkpoints/efficientnet_v2_l-59c71312.pth


  0%|          | 0.00/455M [00:00<?, ?B/s]

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): FusedMBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
      )
      (1): FusedMBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  

In [11]:
def get_feature_vector(image_tensor):
    # a dict to store the activations
    activation = {}
    def getActivation(name):
        # the hook signature
        def hook(model, input, output):
            activation[name] = output.detach()
        return hook

    # register forward hooks on the layers of choice
    h1 = model.avgpool.register_forward_hook(getActivation('avgpool'))

    # forward pass -- getting the outputs
    out = model(image_tensor)

    # detach the hooks
    h1.remove()
    
    feature = torch.squeeze(activation['avgpool'])
    feature = torch.unsqueeze(feature,dim=0)
    return feature

# check
feature = get_feature_vector(image_tensors[0])
print(feature.shape)

torch.Size([1, 1280])
